# Merging, sorting and joining of data

In [ ]:
# Start writing code here...
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Database-style DataFrame or named Series joining/merging
pandas has full-featured, high performance in-memory join operations idiomatically very similar to relational databases like SQL. These methods perform significantly better (in some cases well over an order of magnitude better) than other open source implementations (like base::merge.data.frame in R). The reason for this is careful algorithmic design and the internal layout of the data in DataFrame.


Users who are familiar with SQL but new to pandas might be interested in a comparison with SQL.

pandas provides a single function, merge(), as the entry point for all standard database join operations between DataFrame or named Series objects:  
`pd.merge(
    left,
    right,
    how="inner",
    on=None,
    left_on=None,
    right_on=None,
    left_index=False,
    right_index=False,
    sort=True,
    suffixes=("_x", "_y"),
    copy=True,
    indicator=False,
    validate=None,
)`
* <h5 style="color:red">left:</h5> A DataFrame or named Series object.

* <h5 style="color:red">right:</h5> Another DataFrame or named Series object.

* <h5 style="color:red">on:</h5> Column or index level names to join on. Must be found in both the left and right DataFrame and/or Series objects. If not passed and left_index and right_index are False, the intersection of the columns in the DataFrames and/or Series will be inferred to be the join keys.

* <h5 style="color:red">left_on:</h5> Columns or index levels from the left DataFrame or Series to use as keys. Can either be column names, index level names, or arrays with length equal to the length of the DataFrame or Series.

* <h5 style="color:red">right_on:</h5> Columns or index levels from the right DataFrame or Series to use as keys. Can either be column names, index level names, or arrays with length equal to the length of the DataFrame or Series.

* <h5 style="color:red">left_index:</h5> If True, use the index (row labels) from the left DataFrame or Series as its join key(s). In the case of a DataFrame or Series with a MultiIndex (hierarchical), the number of levels must match the number of join keys from the right DataFrame or Series.

* <h5 style="color:red">right_index:</h5> Same usage as left_index for the right DataFrame or Series

* <h5 style="color:red">how:</h5> One of 'left', 'right', 'outer', 'inner'. Defaults to inner. See below for more detailed description of each method.

* <h5 style="color:red">sort:</h5> Sort the result DataFrame by the join keys in lexicographical order. Defaults to True, setting to False will improve performance substantially in many cases.

* <h5 style="color:red">suffixes:</h5> A tuple of string suffixes to apply to overlapping columns. Defaults to ('_x', '_y').

* <h5 style="color:red">copy:</h5> Always copy data (default True) from the passed DataFrame or named Series objects, even when reindexing is not necessary. Cannot be avoided in many cases but may improve performance / memory usage. The cases where copying can be avoided are somewhat pathological but this option is provided nonetheless.

* <h5 style="color:red">indicator:</h5> Add a column to the output DataFrame called _merge with information on the source of each row. _merge is Categorical-type and takes on a value of left_only for observations whose merge key only appears in 'left' DataFrame or Series, right_only for observations whose merge key only appears in 'right' DataFrame or Series, and both if the observation’s merge key is found in both.

* <h5 style="color:red">validate :</h5> string, default None. If specified, checks if merge is of specified type.

    * “one_to_one” or “1:1”: checks if merge keys are unique in both left and right datasets.

    * “one_to_many” or “1:m”: checks if merge keys are unique in left dataset.

    * “many_to_one” or “m:1”: checks if merge keys are unique in right dataset.

    * “many_to_many” or “m:m”: allowed, but does not result in checks.
`Note:`Support for specifying index levels as the on, left_on, and right_on parameters was added in version 0.23.0. Support for merging named Series objects was added in version 0.24.0.

The return type will be the same as left. If left is a DataFrame or named Series and right is a subclass of DataFrame, the return type will still be DataFrame.

merge is a function in the pandas namespace, and it is also available as a DataFrame instance method merge(), with the calling DataFrame being implicitly considered the left object in the join.

The related join() method, uses merge internally for the index-on-index (by default) and column(s)-on-index join. If you are joining on index only, you may wish to use DataFrame.join to save yourself some typing.

# Brief primer on merge methods (relational algebra)
Experienced users of relational databases like SQL will be familiar with the terminology used to describe join operations between two SQL-table like structures (DataFrame objects). There are several cases to consider which are very important to understand:

`one-to-one` joins: for example when joining two DataFrame objects on their indexes (which must contain unique values).

`many-to-one` joins: for example when joining an index (unique) to one or more columns in a different DataFrame.

`many-to-many` joins: joining columns on columns.

Note:

When joining columns on columns (potentially a many-to-many join), any indexes on the passed DataFrame objects will be discarded.

It is worth spending some time understanding the result of the many-to-many join case. In SQL / standard relational algebra, if a key combination appears more than once in both tables, the resulting table will have the Cartesian product of the associated data. Here is a very basic example with one unique key combination:

In [2]:
left = pd.DataFrame({
   'id':[1,2,3,4,5],
   'Name': ['Alex', 'Amy', 'Allen', 'Alice', 'Ayoung'],
   'subject_id':['sub1','sub2','sub4','sub6','sub5']})
right = pd.DataFrame(
   {'id':[1,2,3,4,5],
   'Name': ['Billy', 'Brian', 'Bran', 'Bryce', 'Betty'],
   'subject_id':['sub2','sub4','sub3','sub6','sub5']})
print( left )
print( right )

   id    Name subject_id
0   1    Alex       sub1
1   2     Amy       sub2
2   3   Allen       sub4
3   4   Alice       sub6
4   5  Ayoung       sub5
   id   Name subject_id
0   1  Billy       sub2
1   2  Brian       sub4
2   3   Bran       sub3
3   4  Bryce       sub6
4   5  Betty       sub5


Merge Two DataFrames on a Key

In [4]:
print( pd.merge(left,right,on='id'))

   id  Name_x subject_id_x Name_y subject_id_y
0   1    Alex         sub1  Billy         sub2
1   2     Amy         sub2  Brian         sub4
2   3   Allen         sub4   Bran         sub3
3   4   Alice         sub6  Bryce         sub6
4   5  Ayoung         sub5  Betty         sub5


Merge Two DataFrames on Multiple Keys

In [6]:
print( pd.merge(left,right,on=['id','subject_id']))

   id  Name_x subject_id Name_y
0   4   Alice       sub6  Bryce
1   5  Ayoung       sub5  Betty


In [13]:
print (pd.merge(left, right, left_on='subject_id',right_on="subject_id"))

   id_x  Name_x subject_id  id_y Name_y
0     2     Amy       sub2     1  Billy
1     3   Allen       sub4     2  Brian
2     4   Alice       sub6     4  Bryce
3     5  Ayoung       sub5     5  Betty


In [15]:
print (pd.merge(left, right,left_index=True,right_index=True))

   id_x  Name_x subject_id_x  id_y Name_y subject_id_y
0     1    Alex         sub1     1  Billy         sub2
1     2     Amy         sub2     2  Brian         sub4
2     3   Allen         sub4     3   Bran         sub3
3     4   Alice         sub6     4  Bryce         sub6
4     5  Ayoung         sub5     5  Betty         sub5


In [18]:
print (pd.merge(left, right,left_index=True,right_on="id"))

   id  id_x  Name_x subject_id_x  id_y Name_y subject_id_y
0   1     2     Amy         sub2     1  Billy         sub2
1   2     3   Allen         sub4     2  Brian         sub4
2   3     4   Alice         sub6     3   Bran         sub3
3   4     5  Ayoung         sub5     4  Bryce         sub6


In [19]:
print (pd.merge(left, right,on="id",sort=True))

   id  Name_x subject_id_x Name_y subject_id_y
0   1    Alex         sub1  Billy         sub2
1   2     Amy         sub2  Brian         sub4
2   3   Allen         sub4   Bran         sub3
3   4   Alice         sub6  Bryce         sub6
4   5  Ayoung         sub5  Betty         sub5


In [21]:
left_ = pd.DataFrame({
   'id':[1,2,3,4,5],
   'Name': ['Alex', 'Amy', 'Allen', 'Alice', 'Ayoung'],
   'subject_id':['sub1','sub3','sub2','sub4','sub6']})
right_ = pd.DataFrame(
   {'id':[1,2,3,4,5],
   'Name': ['Billy', 'Brian', 'Bran', 'Bryce', 'Betty'],
   'subject_id':['sub3','sub2','sub1','sub6','sub5']})

In [26]:
print (pd.merge(left_, right_,on="subject_id",sort=True)) #with sort

   id_x  Name_x subject_id  id_y Name_y
0     1    Alex       sub1     3   Bran
1     3   Allen       sub2     2  Brian
2     2     Amy       sub3     1  Billy
3     5  Ayoung       sub6     4  Bryce


In [27]:
print (pd.merge(left_, right_,on="subject_id"))# with out sort

   id_x  Name_x subject_id  id_y Name_y
0     1    Alex       sub1     3   Bran
1     2     Amy       sub3     1  Billy
2     3   Allen       sub2     2  Brian
3     5  Ayoung       sub6     4  Bryce


In [28]:
print (pd.merge(left, right,left_index=True,right_on="id",suffixes=["_left","_right"])) # by changing suffixes

   id  id_left Name_left subject_id_left  id_right Name_right subject_id_right
0   1        2       Amy            sub2         1      Billy             sub2
1   2        3     Allen            sub4         2      Brian             sub4
2   3        4     Alice            sub6         3       Bran             sub3
3   4        5    Ayoung            sub5         4      Bryce             sub6


### Merge Using 'how' Argument
The how argument to merge specifies how to determine which keys are to be included in the resulting table. If a key combination does not appear in either the left or the right tables, the values in the joined table will be NA.

Here is a summary of the how options and their SQL equivalent names −

|Merge Method|	SQL Equivalent|	Description|
|:---:|:---|---:|
|left|	LEFT OUTER JOIN	|Use keys from left object|
|right|	RIGHT OUTER JOIN|	Use keys from right object|
|outer|	FULL OUTER JOIN|	Use union of keys|
|inner|	INNER JOIN|	Use intersection of keys|


### Left Join

In [7]:
#Left Join
print (pd.merge(left, right, on='subject_id', how='left'))

   id_x  Name_x subject_id  id_y Name_y
0     1    Alex       sub1   NaN    NaN
1     2     Amy       sub2   1.0  Billy
2     3   Allen       sub4   2.0  Brian
3     4   Alice       sub6   4.0  Bryce
4     5  Ayoung       sub5   5.0  Betty


### Right Join

In [8]:
print (pd.merge(left, right, on='subject_id', how='right'))

   id_x  Name_x subject_id  id_y Name_y
0   2.0     Amy       sub2     1  Billy
1   3.0   Allen       sub4     2  Brian
2   NaN     NaN       sub3     3   Bran
3   4.0   Alice       sub6     4  Bryce
4   5.0  Ayoung       sub5     5  Betty


### Outer Join

In [9]:
print (pd.merge(left, right, how='outer', on='subject_id'))

   id_x  Name_x subject_id  id_y Name_y
0   1.0    Alex       sub1   NaN    NaN
1   2.0     Amy       sub2   1.0  Billy
2   3.0   Allen       sub4   2.0  Brian
3   4.0   Alice       sub6   4.0  Bryce
4   5.0  Ayoung       sub5   5.0  Betty
5   NaN     NaN       sub3   3.0   Bran


### Inner Join

In [10]:
print( pd.merge(left, right, on='subject_id', how='inner'))

   id_x  Name_x subject_id  id_y Name_y
0     2     Amy       sub2     1  Billy
1     3   Allen       sub4     2  Brian
2     4   Alice       sub6     4  Bryce
3     5  Ayoung       sub5     5  Betty


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=fcf2399d-084b-4173-af36-20a4a45218a8' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>